In [2]:
import os
import pandas as pd

In [33]:
# @title Helper Functions

def preprocess_leuven_norms(leuven_dir, save_dir, animals, tools):
    '''This function reads the Leuven Norms, selecsts relevant concepts data and saves the dataframes as csv files.'''
    animal_leuven_norms = pd.read_csv(os.path.join(leuven_dir,'ANIMALSexemplarfeaturesbig.txt'), sep = "\t")
    artiacts_leuven_norms = pd.read_csv(os.path.join(leuven_dir,'ARTIFACTSexemplarfeaturesbig.txt'), sep = "\t", encoding='latin-1')


     # in animal norms, the first column is the word, the second column is the frequency, multiply all cells by the freqeuncy 
    animal_leuven_norms.iloc[:,2:] = animal_leuven_norms.iloc[:,2:].multiply(animal_leuven_norms.iloc[:,1], axis=0)
    animal_leuven_norms.iloc[:,2:] = animal_leuven_norms.iloc[:,2:].div(animal_leuven_norms.iloc[:,1].sum(), axis=0)

    artiacts_leuven_norms.iloc[:,2:] = artiacts_leuven_norms.iloc[:,2:].multiply(artiacts_leuven_norms.iloc[:,1], axis=0)
    artiacts_leuven_norms.iloc[:,2:] = artiacts_leuven_norms.iloc[:,2:].div(artiacts_leuven_norms.iloc[:,1].sum(), axis=0)

    # from the third column onwards, replace the values grater than 0 with 1 and 0 otherwise
    animal_leuven_norms.iloc[:,2:] = animal_leuven_norms.iloc[:,2:].applymap(lambda x: 1 if x > 0 else 0)
    artiacts_leuven_norms.iloc[:,2:] = artiacts_leuven_norms.iloc[:,2:].applymap(lambda x: 1 if x > 0 else 0)

    # select the first column and select columns that contain the animal names in the list animals and also the first column
    animal_leuven_norms = animal_leuven_norms[['feature/_exemplar_ENGLISH'] + [col for col in animal_leuven_norms.columns if any(animal in col for animal in animals)]]
    artiacts_leuven_norms = artiacts_leuven_norms[['Item'] + [col for col in artiacts_leuven_norms.columns if any(tool in col for tool in tools)]]

    # rename the first column to 'features'
    animal_leuven_norms.rename(columns={'feature/_exemplar_ENGLISH': 'features'}, inplace=True)
    artiacts_leuven_norms.rename(columns={'Item': 'features'}, inplace=True)

    # transpose the dataframe so that the features are in the rows and the animals are in the columns
    animal_leuven_norms = animal_leuven_norms.T
    artiacts_leuven_norms = artiacts_leuven_norms.T


    # let the first row be the column names and drop the first row
    animal_leuven_norms.columns = animal_leuven_norms.iloc[0]
    animal_leuven_norms = animal_leuven_norms.drop(animal_leuven_norms.index[0])
    # remove the row 'monitor_lizard' from the dataframe
    animal_leuven_norms = animal_leuven_norms.drop('monitor_lizard')

    artiacts_leuven_norms.columns = artiacts_leuven_norms.iloc[0]
    artiacts_leuven_norms = artiacts_leuven_norms.drop(artiacts_leuven_norms.index[0])

    # remove the row 'filing-knife' from the dataframe
    artiacts_leuven_norms = artiacts_leuven_norms.drop('filling-knife')
    artiacts_leuven_norms = artiacts_leuven_norms.drop('pickaxe')

    # save the dataframes as csv files
    animal_leuven_norms.to_csv(os.path.join(save_dir,'animal_leuven_norms.csv'))
    artiacts_leuven_norms.to_csv(os.path.join(save_dir,'artifacts_leuven_norms.csv'))

def load_leuven_norms(save_dir):
    '''This function loads the Leuven Norms dataframes.'''
    animal_leuven_norms = pd.read_csv(os.path.join(save_dir,'animal_leuven_norms.csv'), index_col=0)
    artiacts_leuven_norms = pd.read_csv(os.path.join(save_dir,'artifacts_leuven_norms.csv'), index_col=0)
    return animal_leuven_norms, artiacts_leuven_norms


def load_gpt_data_summer_verified(save_dir):
    '''This function loads the GPT dataframes.'''
    # load from one of the sheets in the excel file
    gpt_response_yes = pd.read_excel(os.path.join(save_dir,'davinci_features_all.xlsx'), sheet_name='all_yes', index_col=0)
    # for the column 'Concept', make the characters lowercase and replace the spaces with underscores
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.lower().str.replace(' ', '_')
    # replace 'boa_python' with 'boa' in the column "Concept"
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('boa_python', 'boa')
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('grinding_disk', 'grinding_disc') 
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('oil_can', 'oilcan')  
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('vacuum', 'vacuum_cleaner')
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('paint_brush', 'paintbrush')
    return gpt_response_yes

def load_gpt_data_summer_unverified(save_dir):
    '''This function loads the GPT dataframes.'''
    # load from one of the sheets in the excel file
    gpt_response_yes_1 = pd.read_excel(os.path.join(save_dir,'davinci_features_all.xlsx'), sheet_name='orig_list_raw')
    # chose columns 'Concept', 'Feature', 'Category' and only unique rows
    # add a column 'Yes/No' with the value 1
    gpt_response_yes_1 = gpt_response_yes_1[['Concept', 'Feature', 'Category']].drop_duplicates()
    gpt_response_yes_1['Yes/No'] = 1
    gpt_response_yes = gpt_response_yes_1
    # # chose columns 'Concept', 'Feature', 'Category' and 'Yes/No'
    # gpt_response_yes_1 = gpt_response_yes_1[['Concept', 'Feature', 'Category', 'Yes/No']]
    # len_1 = len(gpt_response_yes_1)
    # gpt_response_yes_2 = pd.read_excel(os.path.join(save_dir,'davinci_features_all.xlsx'), sheet_name='fv_raw')
    # # chose columns 'Concept', 'Feature', 'Category' and 'Yes/No'
    # gpt_response_yes_2 = gpt_response_yes_2[['Concept', 'Feature', 'Category', 'Yes/No']]
    # len_2 = len(gpt_response_yes_2)
    # gpt_response_yes = pd.concat([gpt_response_yes_1, gpt_response_yes_2], axis=0)
    # # chose only unique rows
    # gpt_response_yes = gpt_response_yes.drop_duplicates()    
    # len_3 = len(gpt_response_yes)
    # assert(len_1+len_2 == len_3)
    # for the column 'Concept', make the characters lowercase and replace the spaces with underscores
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.lower().str.replace(' ', '_')
    # replace 'boa_python' with 'boa' in the column "Concept"
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('boa_python', 'boa')
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('grinding_disk', 'grinding_disc') 
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('oil_can', 'oilcan')  
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('vacuum', 'vacuum_cleaner')
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('paint_brush', 'paintbrush')
    return gpt_response_yes

def load_flan(save_dir, fname):
    '''This function loads the GPT dataframes.'''
    gpt_response_yes_1 = pd.read_excel(os.path.join('../data/feature_listing/summer_data/','davinci_features_all.xlsx'), sheet_name='orig_list_raw')
    gpt_response_yes_1 = gpt_response_yes_1[['Concept', 'Feature', 'Category']].drop_duplicates()
    gpt_response_yes_1['Yes/No'] = 1
    gpt_response_verified = pd.read_csv(os.path.join(save_dir,fname))
    # chose rows where the column 'Yes/No' is 1
    gpt_response_verified = gpt_response_verified[gpt_response_verified['Yes/No'] == str(1)]
    # chose columns 'Concept', 'Feature', 'Category'
    gpt_response_verified = gpt_response_verified[['Concept', 'Feature', 'Category', 'Yes/No']]
    gpt_response_yes = pd.concat([gpt_response_yes_1, gpt_response_verified], axis=0)

    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.lower().str.replace(' ', '_')
    # replace 'boa_python' with 'boa' in the column "Concept"
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('boa_python', 'boa')
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('grinding_disk', 'grinding_disc') 
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('oil_can', 'oilcan')  
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('vacuum', 'vacuum_cleaner')
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('paint_brush', 'paintbrush')
    return gpt_response_yes
     


def load_gpt_data_summer_verified_temperature_0(save_dir):
    gpt_response_yes_1 = pd.read_excel(os.path.join(save_dir,'davinci_features_all.xlsx'), sheet_name='orig_list_raw')
    gpt_response_yes_1 = gpt_response_yes_1[['Concept', 'Feature', 'Category']].drop_duplicates()
    gpt_response_yes_1['Yes/No'] = 1
    gpt_response_verified = pd.read_csv(os.path.join(save_dir,'davinci_feature_listing_feature_list_temperature_0.csv'))
    # chose rows where the column 'Yes/No' is 1
    gpt_response_verified = gpt_response_verified[gpt_response_verified['Yes/No'] == 1.0]
    # chose columns 'Concept', 'Feature', 'Category'
    gpt_response_verified = gpt_response_verified[['Concept', 'Feature', 'Category', 'Yes/No']]
    gpt_response_yes = pd.concat([gpt_response_yes_1, gpt_response_verified], axis=0)

    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.lower().str.replace(' ', '_')
    # replace 'boa_python' with 'boa' in the column "Concept"
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('boa_python', 'boa')
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('grinding_disk', 'grinding_disc') 
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('oil_can', 'oilcan')  
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('vacuum', 'vacuum_cleaner')
    gpt_response_yes['Concept'] = gpt_response_yes['Concept'].str.replace('paint_brush', 'paintbrush')
    return gpt_response_yes


def convert_yes_df_to_wide(df, save_dir, save_fname, verified = True, matrix_completion = False):
    '''This function converts a dataframe to wide format. Fill the missing values with 0 and rest with 1'''
    if not matrix_completion:
        if verified:
            # add a column 'Response' with value 'Yes'
            df['Response'] = 1
             # the concept names are in the column 'Concept', the feature names are in the column 'Feature', the response is in the column 'Response'
            df_wide = df.pivot_table(index='Concept', columns='Feature', values='Response', aggfunc='first').fillna(0)
            df_wide.to_csv(os.path.join(save_dir,save_fname))
        else:
            print('unverified')
            # the concept names are in the column 'Concept', the feature names are in the column 'Feature', the response is in the column 'Response'
            df_wide = df.pivot_table(index='Concept', columns='Feature', values='Yes/No', aggfunc='first').fillna(0)
            df_wide.to_csv(os.path.join(save_dir,save_fname))
    return df_wide


def convert_summer_yes_df_to_wide(df, save_dir, verified = True, matrix_completion = False):
    '''This function converts a dataframe to wide format. Fill the missing values with 0 and rest with 1'''
    if not matrix_completion:
        if verified:
            # add a column 'Response' with value 'Yes'
            df['Response'] = 1
             # the concept names are in the column 'Concept', the feature names are in the column 'Feature', the response is in the column 'Response'
            df_wide = df.pivot_table(index='Concept', columns='Feature', values='Response', aggfunc='first').fillna(0)
            df_wide.to_csv(os.path.join(save_dir,'gpt_response_yes_wide_no_matrix_completion.csv'))
        else:
            print('unverified')
            # the concept names are in the column 'Concept', the feature names are in the column 'Feature', the response is in the column 'Response'
            df_wide = df.pivot_table(index='Concept', columns='Feature', values='Yes/No', aggfunc='first').fillna(0)
            df_wide.to_csv(os.path.join(save_dir,'gpt_response_unverified_wide_no_matrix_completion.csv'))
    return df_wide


# cogsci

In [10]:
leuven_dir = '../../data/LeuvenNorms'
save_dir = '../data/feature_listing'
animals = ['turtle', 'alligator', 'lizard', 'tortoise', 'cobra', 'snake', 'blindworm', 'gecko', 'boa', 'toad', 'crocodile', 'chameleon', 'caiman', 'salamander', 'dinosaur']
tools = ['hammer', 'screwdriver', 'grinding_disc', 'vacuum_cleaner', 'spanner', 'lawn_mower', 'axe', 'saw', 'knife', 'nail', 'chisel', 'shovel', 'anvil', 'oilcan', 'paintbrush']
preprocess_leuven_norms(leuven_dir, save_dir, animals, tools)
animal_leuven_norms, artiacts_leuven_norms = load_leuven_norms(save_dir)

In [32]:
summer_gpt_dir = '../data/feature_listing/summer_data'
gpt_response_yes = load_gpt_data_summer_verified_temperature_0(summer_gpt_dir)
convert_summer_yes_df_to_wide(gpt_response_yes, save_dir, verified=True, matrix_completion = False)

Feature,are_4.5_inches_in_diameter,are_Abrasive,are_Coated,are_Dense,are_Hard,are_Rigid,are_a_constrictor,are_a_good_swimmer,are_a_hand_tool,are_a_hard_plastic,...,lives_in_Central_America,lives_in_Europe,lives_in_South_America,lives_in_different_climates,lives_in_rainforests_of_South_America,lives_in_tropical_climate,lives_in_various_habitats,lives_in_warm_climate,lives_underground,made_of_aluminum_oxide
Concept,,,,,,,,,,,,,,,,,,,,,
alligator,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
anvil,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
axe,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
blindworm,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
boa,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
caiman,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
chameleon,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
chisel,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
cobra,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0


In [12]:
summer_gpt_dir = '../data/feature_listing/summer_data'
gpt_response_unverified_summer = load_gpt_data_summer_unverified(summer_gpt_dir)
convert_summer_yes_df_to_wide(gpt_response_unverified_summer, save_dir, verified=False, matrix_completion = False)

unverified


Feature,are_4.5_inches_in_diameter,are_Abrasive,are_Coated,are_Dense,are_Hard,are_Rigid,are_a_constrictor,are_a_good_swimmer,are_a_hand_tool,are_a_hard_plastic,...,lives_in_Central_America,lives_in_Europe,lives_in_South_America,lives_in_different_climates,lives_in_rainforests_of_South_America,lives_in_tropical_climate,lives_in_various_habitats,lives_in_warm_climate,lives_underground,made_of_aluminum_oxide
Concept,,,,,,,,,,,,,,,,,,,,,
alligator,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
anvil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
axe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
blindworm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
boa,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
caiman,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chameleon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
chisel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cobra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# iclr

In [36]:
data_dir = '../../results/reptile_tool'
save_dir = '../data/feature_listing'
flan = load_flan(data_dir, 'flan_feature_listing_feature_list_temperature_0.csv')
convert_yes_df_to_wide(flan, save_dir, save_fname = 'flan_wide.csv', verified = True, matrix_completion = False)

Feature,are_4.5_inches_in_diameter,are_Abrasive,are_Coated,are_Dense,are_Hard,are_Rigid,are_a_constrictor,are_a_good_swimmer,are_a_hand_tool,are_a_hard_plastic,...,lives_in_Central_America,lives_in_Europe,lives_in_South_America,lives_in_different_climates,lives_in_rainforests_of_South_America,lives_in_tropical_climate,lives_in_various_habitats,lives_in_warm_climate,lives_underground,made_of_aluminum_oxide
Concept,,,,,,,,,,,,,,,,,,,,,
alligator,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
anvil,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
axe,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
blindworm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
boa,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
caiman,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
chameleon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
chisel,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
cobra,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
